# Mixture of Linear Regressions

In this note, we will solve the mixture of linear regressions problem using the moment method.

This block contains some $\LaTeX$ macros.
$\newcommand{\E}{\mathbb{E}}$
$\renewcommand{\Re}{\mathbb{R}}$
$\newcommand{\oft}[1]{{^{(#1)}}}$
$\newcommand{\oftt}[2]{{^{(#1)#2}}}$

In [43]:
import util
from util import *
import models; reload(models)
from models import LinearRegressionsMixture
import numpy as np
np.set_printoptions(precision=3, suppress=True)
from IPython.display import display, Markdown, Math
from operator import mul
import sympy as sp
sp.init_printing()

Let $w_1, ..., w_K \in \Re^D$ be a set of $K$ regression coefficients. Let $x_1, ..., x_N \in \Re^D$ be a given set of data points and let $y_n = \sum_{k} \delta_k w_k^\top x_n + \epsilon$ be observed responses. Our objective is to recover $(w_k)$.

## Toy Example

Let's construct a toy example for the rest of this document.

In [44]:
K, D = 2, 2
np.random.seed(0)
pis = np.array([0.4, 0.6])
ws = np.array([[0.75, 0.25], [0.4, 0.9]])
model = LinearRegressionsMixture.generate("tmp.dat", K, D, betas = ws, weights = pis, cov =  1.0 * np.eye(D))
print("True parameters:")
print(model.weights)
print(model.betas)
print(model.sigma)

True parameters:
[ 0.4  0.6]
[[ 0.75  0.25]
 [ 0.4   0.9 ]]
[[ 1.  0.]
 [ 0.  1.]]


# Noiseless, Infinite Data Setting

In this scenario, assume that $\epsilon = 0$ and that we observe expected moments of the data. Consider the moments:
\begin{align}
\E[x^\alpha y^b] 
&= \sum_k \pi_k \E[x^\alpha y^b | \delta_k = 1] \\
&= \sum_k \pi_k \E[x^\alpha [w_k^\top x]^b] \\
&= \sum_k \pi_k \E[x^\alpha \sum_{\beta \in p(b)} w_k^\beta x^\beta] \\
&= \sum_{\beta \in p(b)} \E_\pi[w^\beta] \E[x^{\alpha + \beta}],
\end{align}
where $p(b)$ are the $d$-partitions of $b$. Note that $\E[x^{\alpha + \beta}]$ are observable quantities, and thus this simply represents a mixture over the polynomials with terms $w_k^\beta$.

In [45]:
def evaluate_mixture(ws, pis, beta):
    """
    Compute E_\pi[w^beta]
    """
    return sum(pi * util.monomial(w, beta) for w, pi in zip(ws.T, pis))
    
def compute_exact_y_moments(ws, pis, moments_x, alpha, b):
    """
    Compute the exact moments E[x^a y^b] using coefficients ws, pis and moments_x.
    """
    D, K = ws.shape
    ret = 0.
    for beta in partitions(D, b):
        ret += evaluate_mixture(ws, pis, beta) * moments_x[tuple_add(alpha, beta)]
    return ret

def describe_moment_polynomial(R, moments_x, moment_y, alpha, b):
    """
    Computes the moment polynomial for E[x^alpha, y^b]
    """
    D = len(R.symbols)
    w = R.symbols
    expr = -moment_y
    for beta in partitions(D, b):
        expr += util.monomial(w, beta) * moments_x[tuple_add(alpha, beta)]
    return expr

In [46]:
# Example 
def double_factorial(n): 
    return reduce(mul, xrange(n, 0, -2)) if n > 0 else 1
def gaussian_moments(sigma, d):
    """
    E[x^d] where x is standard gaussian with sigma
    """
    if d == 0: return 1
    elif d % 2 == 0: return double_factorial(d-1) * sigma**d
    else: return 0
def expected_gaussian_moments(sigma, alphas):
    return {alpha : prod(gaussian_moments(sigma, a) for a in alpha) for alpha in alphas}
def expected_uniform_moments(alphas):
    return {alpha : 1. for alpha in alphas}
def expected_moments_y(ws, pis, moments_x, alphabs):
    return {(alpha, b) : compute_exact_y_moments(ws, pis, moments_x, alpha, b) for alpha, b in alphabs}

In [76]:
R, _ = sp.xring(['w%d'%d for d in xrange(D)], sp.RR, sp.grevlex)

deg_b, deg_x = 2, 2
sigma = model.sigma[0,0]
alphas = list(dominated_elements((deg_x for _ in xrange(D))))
alphabs = [(alpha, b) for alpha in alphas for b in xrange(1,deg_b+1)]
alphas_ = list(dominated_elements((deg_x + deg_b for _ in xrange(D))))
moments_x = expected_gaussian_moments(sigma, alphas_)
moments_y = expected_moments_y(ws, pis, moments_x, alphabs)
#display(moments)

In [77]:
display(moments_x)

In [78]:
display(moments_y)

In [79]:
def get_constraint_polynomials(moments_y, moments_x, deg_x, deg_b):
    constrs = []
    for b in xrange(1, deg_b+1):
        for alpha in util.dominated_elements((deg_x for _ in xrange(D))):
            constrs.append( describe_moment_polynomial(R, moments_x, moments_y[(alpha, b)], alpha, b) )
    return constrs

With this machinery, we can compute the moment polynomials required for the moment method magic!

In [80]:
from mompy.core import MomentMatrix
import mompy.solvers as solvers; reload(solvers)
import mompy.extractors as extractors; reload(extractors)

constrs = get_constraint_polynomials(moments_y, moments_x, deg_x, deg_b)
M = MomentMatrix(3, R.symbols, morder='grevlex')
sol = solvers.solve_generalized_mom(M, constrs, 3)
display( model.betas)
display(extractors.extract_solutions_lasserre(M, sol['x'], Kmax=2))

display(extractors.extract_solutions_dreesen_proto(M, sol['x'], Kmax=2))


     pcost       dcost       gap    pres   dres
 0:  2.4429e-01 -1.7098e+00  3e+01  4e+00  8e+00
 1:  2.4747e-01 -2.8710e+00  6e+00  7e-01  1e+00
 2:  1.3657e-01 -8.2651e-01  1e+00  1e-01  2e-01
 3:  2.4829e-02 -9.0507e-02  1e-01  6e-03  1e-02
 4:  3.0226e-03 -1.3927e-02  2e-02  6e-04  1e-03
 5:  6.6673e-04 -1.3992e-03  2e-03  6e-05  1e-04
 6:  2.1497e-04 -6.3783e-05  3e-04  1e-06  2e-06
 7:  5.4113e-05  6.4174e-06  5e-05  2e-07  3e-07
 8:  3.8111e-05  2.4401e-05  1e-05  1e-15  2e-18
 9:  2.9192e-05  2.7811e-05  1e-06  1e-15  3e-18
10:  2.8475e-05  2.8302e-05  2e-07  1e-15  5e-19
11:  2.8388e-05  2.8368e-05  2e-08  2e-15  5e-19
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.4429e-01 -1.7098e+00  3e+01  4e+00  8e+00
 1:  2.4747e-01 -2.8710e+00  6e+00  7e-01  1e+00
 2:  1.3657e-01 -8.2651e-01  1e+00  1e-01  2e-01
 3:  2.4829e-02 -9.0507e-02  1e-01  6e-03  1e-02
 4:  3.0226e-03 -1.3927e-02  2e-02  6e-04  1e-03
 5:  6.6673e-04 -1.3992e-03  2e-03  6e-05  1e-0

array([[ 0.75,  0.25],
       [ 0.4 ,  0.9 ]])

lost 0.0014679


the next biggest eigenvalue we are losing is 0.001468


In [81]:
display(constrs)

## With real samples

Let's try to solve the problem with generated samples.


In [82]:
def compute_expected_moments(xs, alphas):
    moments = {alpha : 0. for alpha in alphas}
    for alpha in alphas:
        m = monomial(xs.T, alpha)
        moments[alpha] = m if isinstance(m,float) else float(m.mean())
    return moments

def compute_expected_moments_y(ys, xs, alphabs):
    moments = {(alpha, b) : 0. for alpha, b in alphabs}
    for alpha, b in alphabs:
        moments[(alpha,b)] = float((monomial(xs.T, alpha) * ys**b).mean())
        #moments[(alpha,b)] += (monomial(x, alpha) * y**b - moments[(alpha,b)])/(i+1)
    return moments

In [83]:
ys, xs = model.sample(1e6)
moments_x = compute_expected_moments(xs, alphas_)
moments_y = compute_expected_moments_y(ys, xs, alphabs)

In [84]:
moments_y

In [85]:
print(model.sigma)
print sqrt(0.1)
print(xs.T.dot(xs)/1e5)

[[ 1.  0.]
 [ 0.  1.]]
0.316227766017
[[ 9.993  0.021]
 [ 0.021  9.98 ]]


In [86]:

from mompy.core import MomentMatrix
import mompy.solvers as solvers; reload(solvers)
import mompy.extractors as extractors; reload(extractors)

constrs_sdp = get_constraint_polynomials(moments_y, moments_x, deg_x, deg_b)
M = MomentMatrix(3, R.symbols, morder='grevlex')
solsdp = solvers.solve_generalized_mom(M, constrs_sdp, None)
#sol = solvers.solve_moments_with_convexiterations(M, constrs, 3)
display(model.betas)
display(extractors.extract_solutions_lasserre(M, solsdp['x'], Kmax=2))

display(extractors.extract_solutions_dreesen_proto(M, solsdp['x'], Kmax=2))
sol = extractors.extract_solutions_dreesen_proto(M, solsdp['x'], Kmax=2)
ws_rec = array([solsdp[sp.symbols('w0')], solsdp[sp.symbols('w1')]])

     pcost       dcost       gap    pres   dres
 0:  3.0727e-01 -1.6354e+00  3e+01  4e+00  8e+00
 1:  3.1104e-01 -2.8209e+00  6e+00  7e-01  1e+00
 2:  1.7891e-01 -7.9898e-01  1e+00  1e-01  2e-01
 3:  4.5928e-02 -7.8319e-02  1e-01  7e-03  1e-02
 4:  1.4371e-02 -8.0218e-03  2e-02  1e-03  2e-03
 5:  9.7970e-03  3.4212e-03  6e-03  3e-04  5e-04
 6:  9.5163e-03  7.5968e-03  2e-03  5e-05  9e-05
 7:  8.9542e-03  8.3934e-03  6e-04  1e-05  2e-05
 8:  8.7489e-03  8.6987e-03  5e-05  2e-07  3e-07
 9:  8.7118e-03  8.7039e-03  8e-06  2e-08  4e-08
10:  8.7079e-03  8.7050e-03  3e-06  5e-09  8e-09
11:  8.7053e-03  8.7052e-03  1e-07  1e-10  3e-10
12:  8.7053e-03  8.7053e-03  1e-08  1e-11  2e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  3.0727e-01 -1.6354e+00  3e+01  4e+00  8e+00
 1:  3.1104e-01 -2.8209e+00  6e+00  7e-01  1e+00
 2:  1.7891e-01 -7.9898e-01  1e+00  1e-01  2e-01
 3:  4.5928e-02 -7.8319e-02  1e-01  7e-03  1e-02
 4:  1.4371e-02 -8.0218e-03  2e-02  1e-03  2e-0

array([[ 0.75,  0.25],
       [ 0.4 ,  0.9 ]])

lost 0.0002286


the next biggest eigenvalue we are losing is 0.000229


the next biggest eigenvalue we are losing is 0.000229


KeyError: w0

In [87]:
for c1, c2 in zip(constrs,constrs_sdp):
    display(c1,"vs", c2)

'vs'

'vs'

'vs'

'vs'

'vs'

...

'vs'

'vs'

'vs'

'vs'

'vs'

In [14]:
import scipy as sc
import numpy.linalg
np.linalg.matrix_rank((M.numeric_instance(solsdp['x'])), 1e-2)

In [15]:
M.row_monos

In [16]:
M.numeric_instance(solsdp['x'])

array([[   1.   ,    0.487,    0.666,    0.369,    0.458,    0.596,
           0.187,    0.365,    0.496,    0.462],
       [   0.487,    0.369,    0.458,    0.187,    0.365,    0.496,
           2.651,    1.46 ,    1.029,    1.094],
       [   0.666,    0.458,    0.596,    0.365,    0.496,    0.462,
           1.46 ,    1.029,    1.094,    4.054],
       [   0.369,    0.187,    0.365,    2.651,    1.46 ,    1.029,
         -28.06 ,  -11.55 ,   -4.446,   -1.487],
       [   0.458,    0.365,    0.496,    1.46 ,    1.029,    1.094,
         -11.55 ,   -4.446,   -1.487,   -2.854],
       [   0.596,    0.496,    0.462,    1.029,    1.094,    4.054,
          -4.446,   -1.487,   -2.854,  -49.636],
       [   0.187,    2.651,    1.46 ,  -28.06 ,  -11.55 ,   -4.446,
         367.748,  157.181,   67.581,   29.682],
       [   0.365,    1.46 ,    1.029,  -11.55 ,   -4.446,   -1.487,
         157.181,   67.581,   29.682,   16.333],
       [   0.496,    1.029,    1.094,   -4.446,   -1.487,   -2.8

In [17]:
moments_y

In [18]:
moments_x_true = expected_gaussian_moments(sigma, alphas_)
moments_y_true = expected_moments_y(ws, pis, moments_x, alphabs)

In [19]:
for key in moments_y:
    display(key, abs(moments_y[key] - moments_y_true[key]))